In [4]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_7JLSus_U3m4SxY6snjBuCB5KAqBXGMm2h5YrZkSicYdCqQhVwDCNVGrybyf26H8MQVDwTa")

In [5]:
index_name = "user-posts"

In [6]:
pinecone_index = pc.Index(index_name)

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': '33b89cee1155d966b4dbd555ccce715d', 'date': 'Wed, 09 Apr 2025 20:15:47 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"NOT_FOUND","message":"Resource user-posts not found"},"status":404}


In [7]:
# Create index if not exists
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=1024,  # Match Bedrock embedding dimensions
        metric="cosine",
        spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
    )
pinecone_index = pc.Index(index_name)

In [8]:
import streamlit as st
import random
import time
import pandas as pd
import os
from urllib.parse import parse_qs
import boto3
from langchain.llms.bedrock import Bedrock
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import json
from dotenv import load_dotenv

load_dotenv()

session = boto3.Session(profile_name="personal", region_name="us-east-1")
bedrock = boto3.client("bedrock-runtime")

In [9]:
import json
from langchain_text_splitters import RecursiveCharacterTextSplitter

def process_text_file(file_path, useremail):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.create_documents([text])
    
    embeddings = []
    for chunk in chunks:
        body = json.dumps({"inputText": chunk.page_content})
        
        response = bedrock.invoke_model(
            body=body,
            modelId="amazon.titan-embed-text-v2:0",
            accept="application/json"
        )
        
        # Fixed response parsing
        response_body = json.loads(response['body'].read())
        if 'embedding' in response_body:
            embeddings.append(response_body['embedding'])
    
    vectors = [{
        "id": str(hash(chunk.page_content)),
        "values": embedding,
        "metadata": {
            "user_email": useremail,
            "text": chunk.page_content,
            "source": file_path
        }
    } for chunk, embedding in zip(chunks, embeddings)]
    
    return vectors  # Now returns list of vectors

    # pinecone_index.upsert(vectors)


In [11]:
vectors = process_text_file("linkedin_posts (2).txt", useremail="example@example.com")

In [12]:
pinecone_index.upsert(vectors)

{'upserted_count': 79}

In [ ]:
import json
import boto3
import os
from pinecone import Pinecone
from langchain_text_splitters import RecursiveCharacterTextSplitter
from urllib.parse import unquote_plus

# Initialize clients outside handler for cold start optimization
pc = Pinecone(api_key="pcsk_7JLSus_U3m4SxY6snjBuCB5KAqBXGMm2h5YrZkSicYdCqQhVwDCNVGrybyf26H8MQVDwTa")
s3 = boto3.client('s3')
bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')

def lambda_handler(event, context):
    try:
        # Extract bucket and object key from S3 event
        bucket = event['Records'][0]['s3']['bucket']['name']
        key = unquote_plus(event['Records'][0]['s3']['object']['key'])
        
        # Extract user_email from S3 key path (format: user_email/file.txt)
        user_email = key.split('/')[0]
        
        # Download file from S3
        response = s3.get_object(Bucket=bucket, Key=key)
        text = response['Body'].read().decode('utf-8')
        
        # Process text chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, 
            chunk_overlap=200
        )
        chunks = text_splitter.split_text(text)
        
        vectors = []
        for chunk in chunks:
            # Generate embedding using Bedrock
            response = bedrock.invoke_model(
                body=json.dumps({"inputText": chunk}),
                modelId="amazon.titan-embed-text-v2:0",
                accept="application/json"
            )
            response_body = json.loads(response['body'].read())
            
            if 'embedding' not in response_body:
                raise ValueError("Embedding not found in Bedrock response")
                
            vectors.append({
                "id": f"{user_email}-{hash(chunk)}",
                "values": response_body['embedding'],
                "metadata": {
                    "user_email": user_email,
                    "text": chunk,
                    "source": f"s3://{bucket}/{key}"
                }
            })
        
        # Upsert to Pinecone
        pinecone_index = pc.Index("user-posts")
        pinecone_index.upsert(vectors=vectors)
        
        return {
            'statusCode': 200,
            'body': json.dumps({
                'message': 'Successfully processed file',
                'vectors_upserted': len(vectors),
                's3_location': f"s3://{bucket}/{key}"
            })
        }
        
    except Exception as e:
        print(f"Error processing file: {str(e)}")
        return {
            'statusCode': 500,
            'body': json.dumps({
                'error': str(e),
                's3_event': event
            })
        }
